<a href="https://colab.research.google.com/github/spatial-intelligence/F21EP/blob/main/Week1/F21EP_FastAPI_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API demo
Simple demo to run some services on an API - get a counter value, add to the counter, and reset the counter value to 0

In [1]:
# Import necessary libraries
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import threading
import time
import subprocess
import sys

# Enable nested asyncio (required for Colab)
nest_asyncio.apply()

# Create FastAPI app
app = FastAPI(
    title="Simple Counter API",
    description="A simple counter that you can get and add values to",
    version="1.0.0"
)

# Global counter variable
counter = 0

# Pydantic model for adding to counter
class CounterAdd(BaseModel):
    value: int

# Root endpoint
@app.get("/")
async def root():
    """Root endpoint with API info."""
    return {
        "message": "Simple Counter API",
        "current_counter": counter,
        "endpoints": {
            "GET /counter": "Get current counter value",
            "POST /counter/add": "Add a value to the counter",
            "POST /counter/reset": "Reset counter to 0"
        }
    }

# Get current counter value
@app.get("/counter")
async def get_counter():
    """Get the current counter value."""
    return {
        "counter": counter
    }

# Add value to counter
@app.post("/counter/add")
async def add_to_counter(data: CounterAdd):
    """Add a value to the counter."""
    global counter
    counter += data.value
    return {
        "new_value": counter,
    }

# Reset counter
@app.post("/counter/reset")
async def reset_counter():
    """Reset the counter to 0."""
    global counter
    old_value = counter
    counter = 0
    return {
        "old_value": old_value,
        "new_value": counter,
        "message": "Counter has been reset to 0"
    }

# Function to run the server
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

# Start the server in a separate thread
print("Starting FastAPI server on port 8000...")
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(3)

print("Downloading cloudflared...")
# Download cloudflared with proper error handling
try:
    subprocess.run(
        "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /tmp/cloudflared && chmod +x /tmp/cloudflared",
        shell=True,
        check=True,
        capture_output=True
    )
    print("✅ Cloudflared downloaded successfully")
except Exception as e:
    print(f"❌ Download failed: {e}")
    sys.exit(1)

print("Starting Cloudflare Tunnel...\n")

# Start cloudflared tunnel
try:
    tunnel_process = subprocess.Popen(
        ['/tmp/cloudflared', 'tunnel', '--url', 'http://localhost:8000'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    # Read output and find the public URL
    public_url = None
    for line in tunnel_process.stdout:
        print(line.strip())
        if "https://" in line and "trycloudflare.com" in line:
            # Extract the URL
            parts = line.split()
            for part in parts:
                if "https://" in part and "trycloudflare.com" in part:
                    public_url = part.strip()
                    break
            if public_url:
                break

    # If we found the URL, display it nicely
    if public_url:
        print("\n" + "="*70)
        print("🌍 PUBLIC URL READY!")
        print("="*70)
        print(f"\n✅ Your API is now publicly accessible at:\n")
        print(f"   {public_url}\n")
        print("="*70)
        print("\nAvailable endpoints:")
        print(f"  GET  {public_url}/counter")
        print(f"  POST {public_url}/counter/add")
        print(f"  POST {public_url}/counter/reset")
        print(f"  GET  {public_url}/docs (Swagger UI)\n")
        print("="*70)
        print("\nCURL Examples:")
        print(f"\n# Get counter:")
        print(f'curl -X GET "{public_url}/counter"\n')
        print(f"# Add 5 to counter:")
        print(f'curl -X POST "{public_url}/counter/add" \\')
        print(f'  -H "Content-Type: application/json" \\')
        print(f'  -d \'{{"value": 5}}\'\n')
        print(f"# Reset counter:")
        print(f'curl -X POST "{public_url}/counter/reset"\n')
        print("="*70)

    print("\n✨ Keep this cell running to maintain the tunnel!")
    print("="*70)

    # Keep the tunnel alive
    tunnel_process.wait()

except KeyboardInterrupt:
    print("\n\nShutting down...")
    tunnel_process.terminate()
    print("Tunnel closed.")
except Exception as e:
    print(f"Error starting tunnel: {e}")
    sys.exit(1)

Starting FastAPI server on port 8000...
✅ Cloudflared downloaded successfully
Starting Cloudflare Tunnel...

2025-11-21T18:23:05Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-21T18:23:05Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-21T18:23:09Z INF +--------------------------------------------------------------------------------------------+
2025-11-21T18:23:09Z INF |  Your quick Tunnel has been created! Visit it at (it may take som